In [25]:
%matplotlib inline

In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
import joblib
import seaborn as sn

In [27]:
df = pd.read_csv("dataset.csv")
df

,Text,language
0,klement gottwaldi surnukeha palsameeriti ning ...,Estonian
1,sebes joseph pereira thomas på eng the jesuit...,Swedish
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,Thai
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil
4,de spons behoort tot het geslacht haliclona en...,Dutch
...,...,...
21995,hors du terrain les années et sont des année...,French
21996,ใน พศ หลักจากที่เสด็จประพาสแหลมมลายู ชวา อินเ...,Thai
21997,con motivo de la celebración del septuagésimoq...,Spanish
21998,年月，當時還只有歲的她在美國出道，以mai-k名義推出首張英文《baby i like》，由...,Chinese


In [28]:
df.language.unique()

array(['Estonian', 'Swedish', 'Thai', 'Tamil', 'Dutch', 'Japanese',
       'Turkish', 'Latin', 'Urdu', 'Indonesian', 'Portugese', 'French',
       'Chinese', 'Korean', 'Hindi', 'Spanish', 'Pushto', 'Persian',
       'Romanian', 'Russian', 'English', 'Arabic'], dtype=object)

In [29]:
# # example of a ordinal encoding
# from numpy import asarray
# from sklearn.preprocessing import OrdinalEncoder
# # define data
# data = asarray([['red'], ['green'], ['blue']])
# print(data)
# # define ordinal encoding
# encoder = OrdinalEncoder()
# # transform data
# result = encoder.fit_transform(data)
# print(result)

In [30]:
df.Text.isna().sum()

0

In [31]:
df.language.isna().sum()

0

In [12]:
# def funct_enc(i):
#     enc = OneHotEncoder(handle_unknown='ignore')
#     enc.fit(i)
#     enc.categories_
#     df_enc = enc.transform(i)
#     df_encode = pd.DataFrame(df_enc.toarray(), columns = enc.get_feature_names_out())
#     return df_encode

In [13]:
# text_encode = funct_enc(df_text)
# text_encode

* Separation des données

In [35]:
X_train, X_test, y_train, y_test = train_test_split(
  df.Text, df.language, test_size=0.3, random_state = 42
)

* Model de classification

In [46]:
c_svm = SVC(gamma=0.001)

In [47]:
pipe = make_pipeline(CountVectorizer(), TfidfTransformer(), c_svm)
pipe

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('tfidftransformer', TfidfTransformer()),
                ('svc', SVC(gamma=0.001))])

In [52]:
pipe.get_params()

{'memory': None,
 'steps': [('countvectorizer', CountVectorizer()),
  ('tfidftransformer', TfidfTransformer()),
  ('svc', SVC(gamma=0.001))],
 'verbose': False,
 'countvectorizer': CountVectorizer(),
 'tfidftransformer': TfidfTransformer(),
 'svc': SVC(gamma=0.001),
 'countvectorizer__analyzer': 'word',
 'countvectorizer__binary': False,
 'countvectorizer__decode_error': 'strict',
 'countvectorizer__dtype': numpy.int64,
 'countvectorizer__encoding': 'utf-8',
 'countvectorizer__input': 'content',
 'countvectorizer__lowercase': True,
 'countvectorizer__max_df': 1.0,
 'countvectorizer__max_features': None,
 'countvectorizer__min_df': 1,
 'countvectorizer__ngram_range': (1, 1),
 'countvectorizer__preprocessor': None,
 'countvectorizer__stop_words': None,
 'countvectorizer__strip_accents': None,
 'countvectorizer__token_pattern': '(?u)\\b\\w\\w+\\b',
 'countvectorizer__tokenizer': None,
 'countvectorizer__vocabulary': None,
 'tfidftransformer__norm': 'l2',
 'tfidftransformer__smooth_idf': T

In [49]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('tfidftransformer', TfidfTransformer()),
                ('svc', SVC(gamma=0.001))])

* Method GridsearchCV

In [69]:
parameters = {
    'svc__gamma': np.logspace(-5, -1, 20)
}

In [70]:
Mgrid = GridSearchCV(pipe, parameters, cv = 5, n_jobs = -1)

In [71]:
# dir(Mgrid)

In [72]:
Mgrid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('countvectorizer', CountVectorizer()),
                                       ('tfidftransformer', TfidfTransformer()),
                                       ('svc', SVC(gamma=0.001))]),
             n_jobs=-1,
             param_grid={'svc__gamma': array([1.00000000e-05, 1.62377674e-05, 2.63665090e-05, 4.28133240e-05,
       6.95192796e-05, 1.12883789e-04, 1.83298071e-04, 2.97635144e-04,
       4.83293024e-04, 7.84759970e-04, 1.27427499e-03, 2.06913808e-03,
       3.35981829e-03, 5.45559478e-03, 8.85866790e-03, 1.43844989e-02,
       2.33572147e-02, 3.79269019e-02, 6.15848211e-02, 1.00000000e-01])})

In [73]:
Mgrid.best_score_

0.9038311688311689

In [74]:
for param_name in sorted(parameters.keys()):
     print("%s: %r" % (param_name, Mgrid.best_params_[param_name]))

svc__gamma: 0.1


In [88]:
joblib.dump(value = Mgrid.best_estimator_, filename = 'langage_detectors.pkl')

['langage_detectors.pkl']

* Chargement du model

In [89]:
Mgrid_ = joblib.load(filename = 'langage_detectors.pkl')

In [90]:
Mgrid_.score(X_test, y_test)

0.9174242424242425

In [91]:
Mgrid_predict = Mgrid_.predict(X_test)

* Metrics de classification

In [92]:
print(metrics.classification_report(y_test, Mgrid_predict))

              precision    recall  f1-score   support

      Arabic       1.00      0.96      0.98       300
     Chinese       0.39      0.97      0.55       291
       Dutch       1.00      0.96      0.98       313
     English       0.79      0.97      0.87       289
    Estonian       1.00      0.90      0.95       308
      French       0.96      0.97      0.97       287
       Hindi       1.00      0.97      0.99       314
  Indonesian       1.00      0.97      0.98       295
    Japanese       0.99      0.34      0.50       302
      Korean       1.00      0.89      0.94       296
       Latin       0.99      0.86      0.92       312
     Persian       1.00      0.97      0.98       299
   Portugese       0.99      0.95      0.97       293
      Pushto       1.00      0.94      0.97       303
    Romanian       1.00      0.96      0.98       291
     Russian       0.99      0.86      0.92       302
     Spanish       1.00      0.96      0.98       287
     Swedish       1.00    

* Inference et Evaluation

In [93]:
Mgrid_.predict(['je vais au marche, il a des problème comme moi. on fini la formation demain. Si Paris gagne la coupe du monde, les français vont ouvrir leurs grandes geules '])

array(['French'], dtype=object)